# Can we train neural networks without gradient descent ?
> If the lottery ticket hypothesis is real, does that mean we can train a neural network without gradient descent? 

- toc: true 
- badges: true
- comments: true
- tags: [ml]

## What's the problem ?

ML models usually are not really capable of predicting how well the data you     
feed them is close to what was in the dataset. It really matters in production 
models as they might make really stupid mistakes just because they are off       
the training set.                             


                     

Let's train a simple mnist model (straight out from pytorch tutorial https://github.com/pytorch/examples/tree/master/mnist)

In [6]:
#collapse
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR
import os


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout2d(0.25)
        self.dropout2 = nn.Dropout2d(0.5)
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output


def train(args, model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % args.log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))


def test(args, model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))


def mnist():
    filename ="mnist_cnn.pt"
    if os.path.exists(filename):
        return
    # Training settings
    parser = argparse.ArgumentParser(description='PyTorch MNIST Example')
    parser.add_argument('--batch-size', type=int, default=64, metavar='N',
                        help='input batch size for training (default: 64)')
    parser.add_argument('--test-batch-size', type=int, default=1000, metavar='N',
                        help='input batch size for testing (default: 1000)')
    parser.add_argument('--epochs', type=int, default=4, metavar='N',
                        help='number of epochs to train (default: 14)')
    parser.add_argument('--lr', type=float, default=1.0, metavar='LR',
                        help='learning rate (default: 1.0)')
    parser.add_argument('--gamma', type=float, default=0.7, metavar='M',
                        help='Learning rate step gamma (default: 0.7)')
    parser.add_argument('--no-cuda', action='store_true', default=False,
                        help='disables CUDA training')
    parser.add_argument('--seed', type=int, default=1, metavar='S',
                        help='random seed (default: 1)')
    parser.add_argument('--log-interval', type=int, default=10, metavar='N',
                        help='how many batches to wait before logging training status')

    parser.add_argument('--save-model', action='store_true', default=True,
                        help='For Saving the current Model')
    args = parser.parse_args()
    use_cuda = not args.no_cuda and torch.cuda.is_available()

    torch.manual_seed(args.seed)

    device = torch.device("cuda" if use_cuda else "cpu")

    kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
    train_loader = torch.utils.data.DataLoader(
        datasets.MNIST('../data', train=True, download=True,
                       transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ])),
        batch_size=args.batch_size, shuffle=True, **kwargs)
    test_loader = torch.utils.data.DataLoader(
        datasets.MNIST(os.path.expanduser('../data'), train=False, transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ])),
        batch_size=args.test_batch_size, shuffle=True, **kwargs)

    model = Net().to(device)
    optimizer = optim.Adadelta(model.parameters(), lr=args.lr)

    scheduler = StepLR(optimizer, step_size=1, gamma=args.gamma)
    for epoch in range(1, args.epochs + 1):
        train(args, model, device, train_loader, optimizer, epoch)
        test(args, model, device, test_loader)
        scheduler.step()

    if args.save_model:
        torch.save(model.state_dict(), filename)
        
# mnist()



In [7]:
#hide
# Notebook specific hack
import sys; sys.argv=['']; del sys
mnist()

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.296684
Train Epoch: 1 [640/60000 (1%)]	Loss: 1.748730
Train Epoch: 1 [1280/60000 (2%)]	Loss: 0.663051
Train Epoch: 1 [1920/60000 (3%)]	Loss: 0.494142
Train Epoch: 1 [2560/60000 (4%)]	Loss: 0.438472
Train Epoch: 1 [3200/60000 (5%)]	Loss: 0.384302
Train Epoch: 1 [3840/60000 (6%)]	Loss: 0.385620
Train Epoch: 1 [4480/60000 (7%)]	Loss: 0.252736
Train Epoch: 1 [5120/60000 (9%)]	Loss: 0.703955
Train Epoch: 1 [5760/60000 (10%)]	Loss: 0.340966
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.289711
Train Epoch: 1 [7040/60000 (12%)]	Loss: 0.392563
Train Epoch: 1 [7680/60000 (13%)]	Loss: 0.314784
Train Epoch: 1 [8320/60000 (14%)]	Loss: 0.232762
Train Epoch: 1 [8960/60000 (15%)]	Loss: 0.190129
Train Epoch: 1 [9600/60000 (16%)]	Loss: 0.094428
Train Epoch: 1 [10240/60000 (17%)]	Loss: 0.107933
Train Epoch: 1 [10880/60000 (18%)]	Loss: 0.208043
Train Epoch: 1 [11520/60000 (19%)]	Loss: 0.194277
Train Epoch: 1 [12160/60000 (20%)]	Loss: 0.165820
Train Epoch: 1 [12800/60000 (

Train Epoch: 2 [44800/60000 (75%)]	Loss: 0.032387
Train Epoch: 2 [45440/60000 (76%)]	Loss: 0.061855
Train Epoch: 2 [46080/60000 (77%)]	Loss: 0.078439
Train Epoch: 2 [46720/60000 (78%)]	Loss: 0.013668
Train Epoch: 2 [47360/60000 (79%)]	Loss: 0.030492
Train Epoch: 2 [48000/60000 (80%)]	Loss: 0.151191
Train Epoch: 2 [48640/60000 (81%)]	Loss: 0.009170
Train Epoch: 2 [49280/60000 (82%)]	Loss: 0.075856
Train Epoch: 2 [49920/60000 (83%)]	Loss: 0.069568
Train Epoch: 2 [50560/60000 (84%)]	Loss: 0.058522
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.063968
Train Epoch: 2 [51840/60000 (86%)]	Loss: 0.246706
Train Epoch: 2 [52480/60000 (87%)]	Loss: 0.026127
Train Epoch: 2 [53120/60000 (88%)]	Loss: 0.089746
Train Epoch: 2 [53760/60000 (90%)]	Loss: 0.099074
Train Epoch: 2 [54400/60000 (91%)]	Loss: 0.059694
Train Epoch: 2 [55040/60000 (92%)]	Loss: 0.091080
Train Epoch: 2 [55680/60000 (93%)]	Loss: 0.007496
Train Epoch: 2 [56320/60000 (94%)]	Loss: 0.132647
Train Epoch: 2 [56960/60000 (95%)]	Loss: 0.071531


Train Epoch: 4 [28800/60000 (48%)]	Loss: 0.007525
Train Epoch: 4 [29440/60000 (49%)]	Loss: 0.001016
Train Epoch: 4 [30080/60000 (50%)]	Loss: 0.200000
Train Epoch: 4 [30720/60000 (51%)]	Loss: 0.112553
Train Epoch: 4 [31360/60000 (52%)]	Loss: 0.027218
Train Epoch: 4 [32000/60000 (53%)]	Loss: 0.007522
Train Epoch: 4 [32640/60000 (54%)]	Loss: 0.014747
Train Epoch: 4 [33280/60000 (55%)]	Loss: 0.012572
Train Epoch: 4 [33920/60000 (57%)]	Loss: 0.012911
Train Epoch: 4 [34560/60000 (58%)]	Loss: 0.013117
Train Epoch: 4 [35200/60000 (59%)]	Loss: 0.007911
Train Epoch: 4 [35840/60000 (60%)]	Loss: 0.011605
Train Epoch: 4 [36480/60000 (61%)]	Loss: 0.119677
Train Epoch: 4 [37120/60000 (62%)]	Loss: 0.019550
Train Epoch: 4 [37760/60000 (63%)]	Loss: 0.036277
Train Epoch: 4 [38400/60000 (64%)]	Loss: 0.061937
Train Epoch: 4 [39040/60000 (65%)]	Loss: 0.034477
Train Epoch: 4 [39680/60000 (66%)]	Loss: 0.004258
Train Epoch: 4 [40320/60000 (67%)]	Loss: 0.089099
Train Epoch: 4 [40960/60000 (68%)]	Loss: 0.016168


Other out of distribution detector have been proposed. Here is a sample of methods:

- Genetic algorithms
- DFO
- Simulated annealing


## Experiments

In [ ]:
def train_ticket(args, model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % args.log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))


def test_ticket(args, model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))


def ticket_finder():
    filename ="ticket_finder.pt"
    if os.path.exists(filename):
        return
    # Training settings
    parser = argparse.ArgumentParser(description='PyTorch MNIST Example')
    parser.add_argument('--batch-size', type=int, default=64, metavar='N',
                        help='input batch size for training (default: 64)')
    parser.add_argument('--test-batch-size', type=int, default=1000, metavar='N',
                        help='input batch size for testing (default: 1000)')
    parser.add_argument('--epochs', type=int, default=4, metavar='N',
                        help='number of epochs to train (default: 14)')
    parser.add_argument('--lr', type=float, default=1.0, metavar='LR',
                        help='learning rate (default: 1.0)')
    parser.add_argument('--gamma', type=float, default=0.7, metavar='M',
                        help='Learning rate step gamma (default: 0.7)')
    parser.add_argument('--no-cuda', action='store_true', default=False,
                        help='disables CUDA training')
    parser.add_argument('--seed', type=int, default=1, metavar='S',
                        help='random seed (default: 1)')
    parser.add_argument('--log-interval', type=int, default=10, metavar='N',
                        help='how many batches to wait before logging training status')

    parser.add_argument('--save-model', action='store_true', default=True,
                        help='For Saving the current Model')
    args = parser.parse_args()
    use_cuda = not args.no_cuda and torch.cuda.is_available()

    torch.manual_seed(args.seed)

    device = torch.device("cuda" if use_cuda else "cpu")

    kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
    train_loader = torch.utils.data.DataLoader(
        datasets.MNIST('../data', train=True, download=True,
                       transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ])),
        batch_size=args.batch_size, shuffle=True, **kwargs)
    test_loader = torch.utils.data.DataLoader(
        datasets.MNIST(os.path.expanduser('../data'), train=False, transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ])),
        batch_size=args.test_batch_size, shuffle=True, **kwargs)

    model = Net().to(device)
    optimizer = TicketFinder(model.parameters())
    for epoch in range(1, args.epochs + 1):
        train_ticket(args, model, device, train_loader, optimizer, epoch)
        test_ticket(args, model, device, test_loader)
        
    if args.save_model:
        torch.save(model.state_dict(), filename)